In [94]:
import pandas as pd 
from glob import glob
import os
import office365
import requests

import warnings
warnings.filterwarnings("ignore")

In [76]:
path = "D:\pnj.com.vn\HuynhTN - Documents\Data\DataBI"
f = glob("Dthu\*csv")
files_Dthu = glob(os.path.join(path, "Dthu\\*.csv"))
files_CP = glob(os.path.join(path, "CP\\*.csv"))
files_Others = glob(os.path.join(path, "Others\\*.csv"))
files_SLBan = glob(os.path.join(path, "SLBan\\*.csv"))
files_TK = glob(os.path.join(path, "TK\\*.csv"))

# Cphi

In [74]:
df_cp = pd.concat([pd.read_csv(file, encoding='utf-8', sep=',', header=0) for file in files_CP])
df_cp["Month year"] = pd.to_datetime(df_cp["Month year"]).dt.to_period('M')

# create MaCH column with condition right 4 chartacters of cost center code if not null else right 4 characters of GL Account Code
df_cp["MaCH"] = df_cp["Cost Center Code"].fillna(df_cp["Plant Code"].astype('str')).astype('int').astype('str').str[-4:] \
    .replace('0000', 'P015').replace('9999', 'P015')


cols = ["Month year", "GL Account Code", "GL LV1 Name", "FM. Loc Amt", "PnL Name", "MaCH"]
gl_code_exl = 64110120 # list of GL Account Code
gl_code_range = [64100000, 64252100]

# filter data cols with gl_code_exl and in range gl_code_range
df_cp = df_cp[cols][(df_cp["GL Account Code"] != gl_code_exl) & (df_cp["GL Account Code"].between(gl_code_range[0], gl_code_range[1]))]

df_cp.head()

,Month year,GL Account Code,GL LV1 Name,FM. Loc Amt,PnL Name,MaCH
0,2022-01,64130110,CP CCDC,-552500.0,Chi phí bán hàng,1013
1,2022-01,64130140,CP CCDC,-4387500.0,Chi phí bán hàng,1013
2,2022-01,64140110,CP KH TSCĐ,-3336389.0,Chi phí bán hàng,1013
3,2022-01,64140130,CP KH TSCĐ,-1533889.0,Chi phí bán hàng,1013
4,2022-01,64110100,CP Nhân viên,-111131428.0,Chi phí bán hàng,1115


In [ ]:
# group by GL LV1 Name and Month year
df_cp[df_cp["Month year"]=="2025-01"].groupby(["GL LV1 Name", "Month year"])["FM. Loc Amt"].sum() / 1e9

GL LV1 Name                Month year
CP CCDC                    2025-01      -0.744826
CP Còn lại                 2025-01      -0.817176
CP KH TSCĐ                 2025-01      -0.211752
CP Marketing & Khuyến mãi  2025-01      -0.007248
CP Nhân viên               2025-01      -8.924364
CP Thuê ngoài              2025-01      -2.867169
CP chuyển tiền             2025-01      -0.529042
CP hành chính              2025-01      -0.442921
CP sửa chữa, bảo trì       2025-01      -1.238189
CP vật liệu phụ            2025-01      -0.493743
Name: FM. Loc Amt, dtype: float64

# Dthu

In [98]:
df = pd.concat([pd.read_csv(file, encoding='utf-8', sep=',', header=0) for file in files_Dthu])
df["Month year"] = pd.to_datetime(df["Month year"]).dt.to_period('M')
lv2 = ["DÂY ĐỒNG HỒ", "MẮT KÍNH", "ĐỒNG HỒ"]
lv3 = ["CAO"]
lv4 = ["TRANH/TƯỢNG/BIỂU TƯỢNG", "VÀNG ÉP SIÊU PNJ", "VÀNG ÉP SIÊU SJC"]
account_code = [51131000, 51132000]

df_tsv = df.copy()
df_tsv = df_tsv[~df_tsv["Product Group 2 Name"].isin(lv2)]
df_tsv = df_tsv[~df_tsv["Product Group 3 Name"].isin(lv3)]
df_tsv = df_tsv[~df_tsv["Product Group 4 Name"].isin(lv4)]
df_tsv = df_tsv[~df_tsv["GL Account Code"].isin(account_code)]

# df_tsv.groupby(by=['Month year'])[["FM 01_Invoices Revenue", "FM 07_Gross Profit"]].sum()

df_vm = df.copy()
df_vm = df_vm[df_vm["Product Group 4 Name"].isin(lv4)]

# df_vm.groupby(by=['Month year'])[["FM 01_Invoices Revenue", "FM 07_Gross Profit"]].sum()

In [99]:
df = pd.read_csv(files_Dthu[-1], encoding='utf-8', sep=',', header=0)
df["Month year"] = pd.to_datetime(df["Month year"]).dt.to_period('M')
lv2 = ["DÂY ĐỒNG HỒ", "MẮT KÍNH", "ĐỒNG HỒ"]
lv3 = ["CAO"]
lv4 = ["TRANH/TƯỢNG/BIỂU TƯỢNG", "VÀNG ÉP SIÊU PNJ", "VÀNG ÉP SIÊU SJC"]
account_code = [51131000, 51132000]
df_tsv = df[~df["Product Group 2 Name"].isin(lv2)][~df["Product Group 3 Name"].isin(lv3)][~df["Product Group 4 Name"].isin(lv4)][~df["GL Account Code"].isin(account_code)]
df_vm = df[df["Product Group 4 Name"].isin(lv4)]

# Dthu, LG TSV
DT_TSV = f'{df_tsv["FM 01_Invoices Revenue"].sum() / 1e9 :.2f}'
LG_TSV = f'{df_tsv["FM 07_Gross Profit"].sum() / 1e9 :.2f}'
print(DT_TSV, LG_TSV)

# Dthu, LG VM
DT_VM = f'{df_vm["FM 01_Invoices Revenue"].sum() / 1e9 :.2f}'
LG_VM = f'{df_vm["FM 07_Gross Profit"].sum() / 1e9 :.2f}'
print(DT_VM, LG_VM)

213.28 50.16
25.34 2.46


In [86]:
col = ["Product Group 2 Name", "Product Group 3 Name", "Product Group 4 Name", "Month year", "Plant Code", "Channel Description", "FM 01_Invoices Revenue", "FM 07_Gross Profit"]
df_tsv = df_tsv[col]
df_tsv.head()

,Product Group 2 Name,Product Group 3 Name,Product Group 4 Name,Month year,Plant Code,Channel Description,FM 01_Invoices Revenue,FM 07_Gross Profit
0,LOẠI KHÁC,MISCELLANEOUS,BÁCH HÓA TỔNG HỢP,2025-01,1013,Retail,29536292.0,5316533.0
1,LOẠI KHÁC,MISCELLANEOUS,BÁCH HÓA TỔNG HỢP,2025-01,1115,Retail,99000.0,17820.0
2,LOẠI KHÁC,MISCELLANEOUS,BÁCH HÓA TỔNG HỢP,2025-01,1115,Retail,-99000.0,-17820.0
3,LOẠI KHÁC,MISCELLANEOUS,BÁCH HÓA TỔNG HỢP,2025-01,1473,Retail,396000.0,71280.0
4,LOẠI KHÁC,MISCELLANEOUS,BÁCH HÓA TỔNG HỢP,2025-01,1473,Retail,-396000.0,-71280.0


In [90]:
df_tsv["Month year"].dt.year.astype('str').unique()

array(['2025'], dtype=object)

In [80]:
# extract year from series
y = pd.to_datetime(df_tsv["Month year"]).dt.year.unique()
y

# filter by year in list
df_tsv[df_tsv["Month year"].str.contains('|'.join(map(str, y)))]
# df_tsv["Month year"].dt.year()
# pd.to_datetime(df_tsv["Month year"].unique(), format='%m-%Y').dt.strftime('%Y')

,Product Group 2 Name,Product Group 3 Name,Product Group 4 Name,Month year,Plant Code,Channel Description,FM 01_Invoices Revenue,FM 07_Gross Profit
0,LOẠI KHÁC,MISCELLANEOUS,BÁCH HÓA TỔNG HỢP,01-2025,1013,Retail,29536292.0,5316533.0
1,LOẠI KHÁC,MISCELLANEOUS,BÁCH HÓA TỔNG HỢP,01-2025,1115,Retail,99000.0,17820.0
2,LOẠI KHÁC,MISCELLANEOUS,BÁCH HÓA TỔNG HỢP,01-2025,1115,Retail,-99000.0,-17820.0
3,LOẠI KHÁC,MISCELLANEOUS,BÁCH HÓA TỔNG HỢP,01-2025,1473,Retail,396000.0,71280.0
4,LOẠI KHÁC,MISCELLANEOUS,BÁCH HÓA TỔNG HỢP,01-2025,1473,Retail,-396000.0,-71280.0
...,...,...,...,...,...,...,...,...
3841,THAU / HỢP KIM,THAU / HỢP KIM,TRANG SỨC KHÔNG GẮN ĐÁ,01-2025,1565,Online,313409.0,NaN
3842,THAU / HỢP KIM,THAU / HỢP KIM,TRANG SỨC KHÔNG GẮN ĐÁ,01-2025,1575,Retail,318182.0,NaN
3843,THAU / HỢP KIM,THAU / HỢP KIM,TRANG SỨC KHÔNG GẮN ĐÁ,01-2025,1634,Online,318182.0,NaN
3844,THAU / HỢP KIM,THAU / HỢP KIM,TRANG SỨC KHÔNG GẮN ĐÁ,01-2025,1634,Retail,318182.0,NaN


In [ ]:
def load_github_files():
    url = 'https://api.github.com/repos/Huynh-Tr/report/contents/Dthu'
    response = requests.get(url)
    if response.status_code == 200:
        files = pd.DataFrame(response.json())
        files['name'] = files['name'].str.replace('.csv', '')
        return files['download_url'].tolist()
    else:
        st.error("Failed to load data from GitHub.")
        return None

In [100]:
def dthu():
    # path = "D:\pnj.com.vn\HuynhTN - Documents\Data\DataBI"
    # files_Dthu = glob(os.path.join(path, "Dthu\\*.csv"))
    # files_Dthu = load_github_files()

    df = pd.concat([pd.read_csv(file, encoding='utf-8', sep=',', header=0) for file in load_github_files()])
    # df["Month year"] = pd.to_datetime(df["Month year"], format="%m-%Y").dt.to_period('M')
    lv2 = ["DÂY ĐỒNG HỒ", "MẮT KÍNH", "ĐỒNG HỒ"]
    lv3 = ["CAO"]
    lv4 = ["TRANH/TƯỢNG/BIỂU TƯỢNG", "VÀNG ÉP SIÊU PNJ", "VÀNG ÉP SIÊU SJC"]
    account_code = [51131000, 51132000]
    col = ["Product Group 2 Name", "Product Group 3 Name", "Product Group 4 Name", "Month year", \
        "Plant Code", "Channel Description", "FM 01_Invoices Revenue", "FM 07_Gross Profit"]

    df_tsv = df.copy()
    df_tsv = df_tsv[~df_tsv["Product Group 2 Name"].isin(lv2)]
    df_tsv = df_tsv[~df_tsv["Product Group 3 Name"].isin(lv3)]
    df_tsv = df_tsv[~df_tsv["Product Group 4 Name"].isin(lv4)]
    df_tsv = df_tsv[~df_tsv["GL Account Code"].isin(account_code)]
    df_tsv = df_tsv[col]

    df_vm = df.copy()
    df_vm = df_vm[df_vm["Product Group 4 Name"].isin(lv4)]
    df_vm = df_vm[col]

    return df_tsv, df_vm

In [101]:
df_tsv, df_vm = dthu()

KeyError: 'Product Group 2 Name'

# Parquet

In [102]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from glob import glob  # For handling multiple CSV files
import os

In [ ]:
path = "D:\pnj.com.vn\HuynhTN - Documents\Project\streamlit"

csv_files = glob(os.path.join(path, "Dthu\\*.csv"))  # Get all CSV files in the directory
# create a directory to save parquet files
if not os.path.exists(os.path.join(path, "\\dthu_parquet_files")):
    os.makedirs(os.path.join(path, "\\dthu_parquet_files"))

# name of file
# file_name = csv_files[0].split("\\")[-1].replace('.csv', '.parquet')
# file_name
# print(os.path.basename(csv_file[0]))

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    table = pa.Table.from_pandas(df)

    # Create parquet filename based on csv filename:
    parquet_file = parquet_directory + "\\" + os.path.basename(csv_file).replace('.csv', '.parquet')

    pq.write_table(table, parquet_file)


FileNotFoundError: [WinError 3] Failed to open local file 'D:/pnj.com.vn/HuynhTN - Documents/Project/streamlit/Dthu/parquet_files/202101.parquet'. Detail: [Windows error 3] The system cannot find the path specified.


In [152]:
# open parquet file use pandas
parquet_files = glob(os.path.join(path, "Dthu\\parquet_files\\*.parquet"))
# parquet_file
df = pd.concat([pd.read_parquet(parquet_file) for parquet_file in parquet_files])
df

,Company Code,Month year,Product Group 2 Code,Product Group 3 Code,Product Group 4 Code,Plant Code,Channel Description,FM 01_Invoices Revenue,FM 07_Gross Profit,Product Group 2 Name,Product Group 3 Name,Product Group 4 Name,GL Account Code
0,1000,Jan 2021,PC,PCM,PCM00,1447,Retail,140000.0,25200.0,LOẠI KHÁC,MISCELLANEOUS,BÁCH HÓA TỔNG HỢP,51111100
1,1000,Jan 2021,PC,PCS,PCS00,1115,Retail,481980.0,86756.0,LOẠI KHÁC,SERVICE,SERVICE,51131000
2,1000,Jan 2021,PC,PCS,PCS00,1255,Retail,852750.0,153495.0,LOẠI KHÁC,SERVICE,SERVICE,51131000
3,1000,Jan 2021,PC,PCS,PCS00,1256,Online,306728.0,55211.0,LOẠI KHÁC,SERVICE,SERVICE,51131001
4,1000,Jan 2021,PC,PCS,PCS00,1256,Retail,3439454.0,619102.0,LOẠI KHÁC,SERVICE,SERVICE,51131000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3896,1000,Jan 2025,PW,PWM,PWMOR,1473,Retail,48822038.0,14646611.0,ĐỒNG HỒ,NAM,ORIENT,51111100
3897,1000,Jan 2025,PW,PWM,PWMTI,1115,Online,13611111.0,4083333.0,ĐỒNG HỒ,NAM,TISSOT,51111100
3898,1000,Jan 2025,PW,PWM,PWMTI,1115,Retail,13029398.0,3908819.0,ĐỒNG HỒ,NAM,TISSOT,51111100
3899,1000,Jan 2025,PW,PWM,PWMTI,1277,Retail,0.0,0.0,ĐỒNG HỒ,NAM,TISSOT,51111100
